In [1]:
import torch
import tiktoken

In [2]:
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)

In [3]:
with open("D:/python/LLMS from scratch/the-verdict.txt", encoding="utf-8") as f:
    raw_text = f.read()

In [4]:
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]
        #print(tokenizer)
        token_ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
        assert len(token_ids)>max_length,"Number of tokenized inputs must atleast be equal to max_length+1"
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]

In [5]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    tokenizer=tiktoken.get_encoding("gpt2")
    #print(tokenizer)
    dataset=GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader=DataLoader(dataset,batch_size=batch_size,drop_last=drop_last,num_workers=num_workers)
    return dataloader

In [6]:
max_length=4
dataloader=create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)

In [7]:
print("Token IDs:\n",inputs)
print("\nInputs shape:\n",inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [8]:
token_embeddings=token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [9]:
print(token_embeddings)

tensor([[[-0.1185,  1.2531,  1.1718,  ...,  0.5579, -0.0919, -1.6358],
         [ 0.8390,  1.2565, -0.2434,  ..., -0.1829,  0.9217, -0.5658],
         [-0.0914,  0.2528, -0.0332,  ...,  1.2087,  1.7421,  1.8649],
         [-0.7077, -1.0134, -0.9225,  ...,  0.8112, -1.1777,  0.7985]],

        [[ 0.5194, -0.4138,  0.7430,  ...,  1.2920,  0.2389,  0.9786],
         [ 1.3507, -0.7826, -1.1391,  ..., -0.2171,  0.1396, -0.1430],
         [-2.3030,  0.0967, -1.6021,  ...,  0.1107, -0.6649, -1.6735],
         [-1.0469, -0.4320, -1.9581,  ...,  0.5492,  0.3595, -0.5028]],

        [[-1.2574,  1.3457,  1.3278,  ..., -0.4519, -0.0385,  0.7265],
         [ 1.0049,  0.4325,  0.9904,  ...,  2.4112, -0.3613, -2.2954],
         [-0.5660,  0.8005, -1.2830,  ..., -0.1358, -0.1082, -1.9270],
         [-0.5165, -1.7819,  1.2953,  ...,  0.6293, -2.3749,  1.0709]],

        ...,

        [[ 0.3189,  1.2414, -0.8044,  ..., -0.4731,  0.3910, -2.9868],
         [-0.9354, -1.4637, -1.0346,  ..., -0.1318, -0.98

In [10]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 0.3399, -0.8767,  0.1429,  ...,  0.7552,  0.7155,  0.1646],
        [-0.6140, -0.7585,  0.3350,  ...,  0.2641,  1.0881,  1.1475],
        [ 0.0867, -1.1406,  0.4970,  ...,  0.6885, -1.0880, -1.0414],
        [-0.7297, -0.5230,  1.1102,  ...,  0.6880,  0.0716, -0.5580]],
       requires_grad=True)


In [12]:
print(pos_embedding_layer)

Embedding(4, 256)
